In [12]:
import vtk
import matplotlib.cm as cm
from vtk.util import numpy_support
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets

In [13]:
def read_vti_file(filename):
    """
    Reads a VTI file using VTK and returns the output as a vtkImageData object.

    Args:
        filename (str): The path to the VTI file.

    Returns:
        vtkImageData: The image data read from the VTI file.
    """
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()

In [14]:
def convert_vtk_to_numpy(vtk_data):
    """
    Converts the scalar field data from a VTK object to a NumPy array.

    Args:
        vtk_data (vtkImageData): The vtkImageData object containing the scalar field.

    Returns:
        numpy.ndarray: The scalar field data as a NumPy array.
    """
    scalar_field = vtk_data.GetPointData().GetScalars()
    scalar_field_array = numpy_support.vtk_to_numpy(scalar_field)
    return scalar_field_array.reshape(vtk_data.GetDimensions())


In [15]:
def create_isosurface_trace(x, y, z, scalar_field_array, isosurface_value, colormap, color_range):
    """
    Creates an isosurface trace for Plotly using the given data and settings.

    Args:
        x, y, z (numpy.ndarray): Coordinates for the 3D points.
        scalar_field_array (numpy.ndarray): The scalar field data.
        isosurface_value (float): The isosurface value to visualize.
        colormap (str or plotly.colors.sequential.Colormap): The colormap for the isosurface.
        color_range (list): The minimum and maximum values for the color scale.

    Returns:
        plotly.graph_objs.Isosurface: The isosurface trace for Plotly.
    """
    isosurface = go.Isosurface(
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        value=scalar_field_array.flatten(),
        isomin=isosurface_value,
        isomax=isosurface_value,
        surface_count=1,
        caps=dict(x_show=False, y_show=False),
        colorscale=colormap,
        colorbar=dict(thickness=20, ticklen=4, len=0.75, title="Density"),
        cmin=color_range[0],
        cmax=color_range[1],
        showscale=True,
    )
    return isosurface

In [16]:
def create_histogram_trace(scalar_field_array, isosurface_value):
    """
    Creates a histogram trace for Plotly of the scalar field centered around the isosurface value.

    Args:
        scalar_field_array (numpy.ndarray): The scalar field data.
        isosurface_value (float): The center value for the histogram bins.

    Returns:
        plotly.graph_objs.Histogram: The histogram trace for Plotly.
    """
    histogram = go.Histogram(
        x=scalar_field_array.flatten(),
        nbinsx=30,
        xbins=dict(start=isosurface_value - 0.25, end=isosurface_value + 0.25),
    )
    return histogram

In [17]:
def create_plot_layout(x_range, y_range, z_range):
    return go.Layout(
        scene=dict(
            xaxis=dict(nticks=4, range=x_range),
            yaxis=dict(nticks=4, range=y_range),
            zaxis=dict(nticks=4, range=z_range),
        ),
    )

In [18]:
def create_histogram_layout():
    return go.Layout(
        xaxis=dict(title="Vortex Scalar Field"),
        yaxis=dict(title="Frequency"),
    )

In [9]:
# ----- Function for Interactive Updates -----
def update_plot(isosurface_fig, histogram_fig, scalar_field_array, slider_value, isosurface_value):
    """
    Updates the isosurface and histogram figures based on the new slider value.

    Args:
        isosurface_fig (plotly.graph_objs.Figure): The isosurface figure.
        histogram_fig (plotly.graph_objs.Figure): The histogram figure.
        scalar_field_array (numpy.ndarray): The scalar field data.
        slider_value (float): The new value from the slider.
        isosurface_value (float): The initial isosurface value.
    """
    with isosurface_fig.batch_update():
        isosurface_fig.data[0].isomin = slider_value
        isosurface_fig.data[0].isomax = slider_value
    with histogram_fig.batch_update():
        if slider_value:
            histogram_fig.data[0].xbins.start = slider_value - 0.25
            histogram_fig.data[0].xbins.end = slider_value + 0.25
        else:
            histogram_fig.data[0].xbins.start = scalar_field_array.min()
            histogram_fig.data[0].xbins.end = scalar_field_array.max()

In [19]:
def reset_plots(r, slider, isosurface_value):
    slider.value = isosurface_value

In [21]:
def main():
    """
    Reads the VTI file, creates the interactive plots, and sets up the controls.
    """
    # Read VTI file
    vtk_data = read_vti_file("mixture.vti")

    # Convert VTK data to numpy array
    scalar_field_array = convert_vtk_to_numpy(vtk_data)

    # Generate coordinates
    x, y, z = np.indices(scalar_field_array.shape)

    # Define isosurface value, colormap, and color range
    # isosurface_value = 0
    # colormap = px.colors.sequential.plasma
    # color_range = [scalar_field_array.min(), scalar_field_array.max()]
    isosurface_value = 0
    # colormap = cm.plasma
    colormap = 'plasma'

    color_range = [scalar_field_array.min(), scalar_field_array.max()]

    # Create isosurface trace
    isosurface_trace = create_isosurface_trace(x, y, z, scalar_field_array, isosurface_value, colormap, color_range)

    # Create histogram trace
    histogram_trace = create_histogram_trace(scalar_field_array, isosurface_value)

    # Create plot layout
    isosurface_layout = create_plot_layout([0, 75], [0, 75], [0, 75])
    histogram_layout = create_histogram_layout()

    # Create Plotly figures
    isosurface_fig = go.FigureWidget(data=[isosurface_trace], layout=isosurface_layout)
    histogram_fig = go.FigureWidget(data=[histogram_trace], layout=histogram_layout)

    # Initialize slider
    slider = widgets.FloatSlider(
        value=isosurface_value,
        min=scalar_field_array.min(),
        max=scalar_field_array.max(),
        step=0.01,
        description="Iso-surface value:",
        continuous_update=True,
    )

    # Initialize reset button
    reset_button = widgets.Button(description="Reset")

    # Bind events
    slider.observe(lambda change: update_plot(isosurface_fig, histogram_fig, scalar_field_array, change["new"], isosurface_value), names="value")
    reset_button.on_click(lambda button_click: reset_plots(button_click, slider, isosurface_value))

    # Display the plots
    display(widgets.VBox([widgets.HBox([isosurface_fig, histogram_fig]), widgets.HBox([slider, reset_button])]))

if __name__ == "__main__":
    main()


    'data': [{'caps': {'x': {'show': False}, 'y': {'show': False}…